#### I. Introducción

* Contexto del problema a resolver y justificación de la necesidad:
    
    El problema a resolver es la incapacidad de extraer información valiosa de una gran fuente de datos como Twitter. La obtención de información relevante es crucial en campos como el marketing, publicidad, política, investigación, atención al cliente, entre otros.

* Objetivos y alcance del proyecto:

    El objetivo del proyecto es analizar y clasificar el texto publicado en la red social Twitter según su sentimiento (positivo o negativo) mediante el uso de técnicas de machine learning. También se pretende acceder a la API de Twitter para analizar en tiempo real el sentimiento que genera en el conjunto de los usuarios un tema en concreto.

#### II. Dataset

* Descripción del dataset utilizado:

    El dataset utilizado contiene información sobre tweets extraídos de Twitter mediante su API en el año 2009 y que han sido clasificados previamente según si su sentimiento era positivo o negativo.
        
    * Origen: https://www.kaggle.com/datasets/kazanova/sentiment140
    * Tamaño: 1600000 tweets.
    * Variables: `target`, `id`, `date`, `flag`, `user`, `text`. De estas variables, únicamente `target` y `text` son relevantes para el análisis de sentimiento.

* Análisis exploratorio de los datos:

    Se ha comprobado que la variable target solo contiene 2 posibles valores que corresponden a los dos sentimientos objeto del análisis, y que la cantidad de registros en cada clase está balanceada. Además, se ha verificado que no hay valores nulos en el dataset.
    
    <img src="../resources/values.png" width="500">

#### III. Preprocesamiento de los datos

Se ha decidido prescindir de la mayoría de las variables presentes en el dataset ya que, pese a poder presentar un alto valor predictivo en el set de entrenamiento, no se ajustan al objetivo que busca este proyecto, que pretende predecir el sentimiento basándose únicamente en el texto, y poder sacar conclusiones sobre la relación que puedan presentar el resto de variables con el sentimiento obtenido y no al revés.

El dataset se ha reducido a una muestra aleatoria de 20,000 tweets para poder trabajar con los datos.

Debido a la naturaleza y particularidades de los textos utilizados en este proyecto, se han considerado irrelevantes ciertos elementos como las menciones a usuarios, las direcciones URL y los emojis que ya habían sido previamente eliminados por el creador del dataset. Se han utilizado expresiones regulares para su eliminación.

También se ha sometido el dataset a un proceso de lematización cuyo objetivo es reducir el ruido agrupando en un lema las palabras que compartan un significado común.

Se han eliminado símbolos mediante una expresión regular que selecciona cualquier carácter no alfanumérico o espacio.

Se ha decidido separar en este punto el set de entrenamiento del set de pruebas para que este último no se vea contaminado por los siguientes procesos.

Con un tokenizer, se divide el texto en palabras y se les asigna un número a cada palabra única, creando así un diccionario de palabras y su frecuencia en el corpus. Se ha limitado el número de palabras que puede tener en cuenta a modo de vocabulario a 3000, lo que debería cubrir alrededor del 90% de las palabras utilizadas.

Mediante un proceso de vectorización y utilizando el tokenizer entrenado con nuestros datos, se genera un vector de números que representa las palabras del texto.

Finalmente, se determina el vector con las dimensiones más grandes y se añaden ceros al principio de cada vector hasta que tenga la longitud máxima, de manera que puedan ser utilizados como entrada para el modelo.

#### IV. Modelado

* Evaluación de los diferentes modelos e iteraciones:

    Inicialmente se probó la combinación de un `RandomForestClassifier` con `SentimentIntensityAnalyzer` utilizando la librería nltk y se obtuvo una precisión del 68%. Debido a la simplicidad de este método y a la poca implicación que requería por mi parte, se decidió tomar otro camino y optar por un método que no utilizara ningún modelo preentrenado o corpus ya clasificado. Finalmente, se llegó a la conclusión de que, tratándose de textos cortos en su mayoría, la mejor opción en este caso es una red neuronal LSTM (Long Short-Term Memory) ya que su capacidad para recordar inputs pasados la hace perfecta para el procesamiento de textos, su único problema sería al recibir un texto de mayores dimensiones.

* Selección e interpretación del modelo final:

    * Como entrada se ha utilizado una capa `Embedding` que ayudará a reducir la dimensionalidad de los vectores que previamente habían sido generados. Utiliza las 3000 palabras descritas anteriormente como vocabulario.
    * Para evitar el sobreajuste, se ha optado por una capa `SpatialDropout1D` que bloquea aleatoriamente algunas de las entradas de la primera capa para fomentar la generalización.
    * Capa `LSTM` de tamaño 256, permite que el modelo recuerde dependencias y que lo utilice para aprender de ellas.
    * Última capa `Dense` de salida binaria, una para cada clase. Con función de activación sigmoide, producirá valores de salida entre 0 y 1 interpretables como la probabilidad de que la entrada del modelo sea de una clase determinada.

    Accuracy:

    <img src="../resources/accuracy.png" width="500">

    Loss:

    <img src="../resources/loss.png" width="500">

    Por las gráficas de accuracy y loss, se puede deducir que el modelo no está generalizando bien y que prácticamente no está aprendiendo. Esto puede significar que los datos utilizados como input son demasiado complejos o que la red neuronal es demasiado simple. Para fomentar la generalización del modelo, se ha aumentado el dropout de la capa `LSTM`, pero sigue sin conseguir unos resultados satisfactorios.

#### V. Predicción y resultados finales

Finalmente se ha conseguido una precisión del 72% que no es un mal rendimiento en la clasificación de tweets en función de su sentimiento. Sin embargo, la precisión no es lo suficientemente alta para ser utilizado de forma fiable en un contexto empresarial o de investigación.



#### VI. Conclusiones y futuros pasos

Se ha conseguido una precisión considerable que puede ser utilizada para la clasificación de tweets en base a su sentimiento.

Como siguientes pasos sería interesante poder aplicar estas clasificaciones a casos reales mediante la utilización de la API de Twitter y poder establecer correlaciones entre los diferentes datos que ofrece.